<a href="https://colab.research.google.com/github/kacper-dobrow/tensorflow-course/blob/main/08_intro_to_NLP_in__TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP fundamentals in TensorFlow

NLP has the goal of deriving information out of natural language (e.g.: sequences of text or speech)

Another common term for NLP problems is sequence to sequence problems (seq2seq)